In [1]:
import os
import logging
from datetime import datetime
import json

import numpy as np
from csbdeep.models import Config
import onnxruntime as ort
import onnx

from flame import FLAMEImage
from flame.utils import min_max_norm
from flame.error import FLAMEImageError

2025-05-15 15:46:07.771047: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 15:46:07.779144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747349167.787753  343478 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747349167.790366  343478 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747349167.797382  343478 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
assert 'CUDAExecutionProvider' in ort.get_available_providers()

In [3]:
INFERENCE_DATA_DIR = "/mnt/d/data/raw/0013_250514_HS6307_CAREtest_NA"
DATASET_DIRECTORY = "/mnt/d/code/Balu_CARE/datasets"
DATASET_NAME = "20250513_40I_denoising_7to40F"
DATASET_JSON = os.path.join(DATASET_DIRECTORY, f"{DATASET_NAME}.json")
MODEL_DIRECTORY = "/mnt/d/models/CARE/test_model"
MODEL_NAME = os.path.basename(MODEL_DIRECTORY)
ONNX_PATH = os.path.join(MODEL_DIRECTORY, f"{MODEL_NAME}.onnx")
JSON_PATH = os.path.join(MODEL_DIRECTORY, f"config.json")

for f in [DATASET_JSON, ONNX_PATH, JSON_PATH]:
    assert os.path.isfile(f)

### Loading Images

In [4]:
images = []
for root, dirs, files in os.walk(INFERENCE_DATA_DIR):
    for f in files:
        if "tif" in f or "tiff" in f:
            try:
                this_image = FLAMEImage(
                    impath = os.path.join(root, f),
                    jsonext = "tileData.txt",
                    overrideNFrames = 1,
                    checkFrames = False,
                    checkZs = True
                )
            except FLAMEImageError as e:
                continue
            images.append(this_image)

In [5]:
images[0].axes_shape

'ZCYX'

In [27]:
test_output_dir = "/mnt/c/Users/BaluLab/Desktop/test"
os.makedirs(test_output_dir, exist_ok=True)

In [6]:
images[0].openImage()

In [ ]:
for idx in range(images[0].imShape[0]):
    tiff.imwrite(os.path.join(test_output_dir, f"{idx}.tiff"), images[0].raw()[idx,...].transpose(1, 2, 0).astype(np.uint8))

In [5]:
model_config_dict = json.load(open(JSON_PATH, 'r'))
dataset_config_dict = json.load(open(DATASET_JSON, 'r'))

In [6]:
dataset_config_dict['FLAME_Dataset'][]

SyntaxError: invalid syntax (2034356163.py, line 1)

### Loading ONNX

In [ ]:
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

In [ ]:
ort_session = ort.InferenceSession(
    ONNX_PATH,
    providers=['CUDAExecutionProvider']
)

2025-05-15 11:31:27.123088841 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-15 11:31:27.123117107 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:
input_tensor = ort_session.get_inputs()[0]
input_name, input_shape, input_type = input_tensor.name, input_tensor.shape, input_tensor.type
print(f"Input Tensor\nName: {input_name}\nShape: {input_shape}\nType: {input_type}")

Input Tensor
Name: patch
Shape: ['unk__193', 128, 128, 3]
Type: tensor(float)
